In [99]:
import pandas as pd
import re
import boto3
import time
import os
import pandas as pd
import uuid  # To generate unique identifiers for job names
import requests
from jiwer import wer

In [110]:
sentences_df = pd.read_csv('../data/cv-corpus-19.0-delta-2024-09-13/en/validated.tsv', delimiter='\t')

In [ ]:
import boto3
import time
import os
import pandas as pd
from jiwer import wer

# Initialize Boto3 clients
s3_client = boto3.client('s3')
transcribe_client = boto3.client('transcribe')
num_samples = 20
# Constants
s3_bucket_name = 'test-transcribe-comparison'  # Replace with your S3 bucket name

def upload_to_s3(file_path, bucket_name):  
    s3_key = os.path.basename(file_path)
    s3_client.upload_file("../data/cv-corpus-19.0-delta-2024-09-13/en/clips/" + file_path, bucket_name, s3_key)
    s3_url = f"s3://{bucket_name}/{s3_key}"
    return s3_url

def start_transcription_job(s3_url, job_name):
    response = transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': s3_url},
        MediaFormat='mp3',
        LanguageCode='en-US'
    )
    return response

def get_transcription_result(job_name):
    # Wait until the job is complete
    while True:
        status = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
        time.sleep(10)
    
    if status['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
        transcription_url = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
        # Fetch the transcription from the URL
        response = requests.get(transcription_url)
        response.raise_for_status()  # Raise an error for bad status codes
        transcript_data = response.json()
        transcript_text = transcript_data['results']['transcripts'][0]['transcript']
        return transcript_text
    else:
        raise Exception(f"Transcription job {job_name} failed")

# Upload files, start transcription jobs, and calculate error rate
error_rates = []
for index, row in sentences_df.head(num_samples).iterrows():
    print("Working on:")
    print(row['sentence'])
    # Extract file path and ground truth
    file_path = row['path']
    ground_truth = row['sentence']
    
    # Upload to S3
    s3_url = upload_to_s3(file_path, s3_bucket_name)
    
    # Start transcription job
    job_name = job_name = f"transcribe_job_{index}_{uuid.uuid4()}"
    start_transcription_job(s3_url, job_name)
    sentences_df.at[index, 'job_name'] = job_name  # Add job name to DataFrame

for index, row in sentences_df.head(num_samples).iterrows():
    job_name = row['job_name']
    # Retrieve transcription result
    transcript_text = get_transcription_result(job_name)
    
    # Calculate error rate
    error_rate = wer(ground_truth, transcript_text)
    error_rates.append((file_path, error_rate))
    print(transcript_text)
    
    print("error rate: " + str(error_rate))

# Convert results to DataFrame for easier review
error_rate_df = pd.DataFrame(error_rates, columns=['File', 'Error Rate'])
print(error_rate_df)


Working on:
The new division adopted the Lazarus nameplate, while assuming Shillito's Rike's headquarters in Cincinnati.
Working on:
He was subsequently assigned to minor posts in Jerusalem, Rome, Cambridge and Strasbourg.
Working on:
The phrase originated in a comic strip of the same name.
Working on:
Gibson went on to manage the Department of Recreation in East Orange, New Jersey.
Working on:
On Sundays, four trains run each way, to Carlisle and Whitehaven only.
Working on:
There is a police station on the main road.
Working on:
Each pattern is available in two variations, labelled "A" and "B".
Working on:
In other respects, however, they have primitive features, closer to nonruminants such as pigs.
Working on:
The most radical change to the Vantage, however, was inside the engine compartment.
Working on:
Fort Stockton and Pecos County are part of the Midland College service area.
Working on:
The historicity of Samyogita is a matter of debate.
Working on:
In some cases, Kansai dialec

In [ ]:
# Convert results to DataFrame for easier review and calculate average WER
error_rate_df = pd.DataFrame(error_rates, columns=['File', 'Error Rate'])
average_wer = error_rate_df['Error Rate'].mean()

print(error_rate_df)
print(f"Average Word Error Rate (WER): {average_wer}")